In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import zipfile
import random
import seaborn as sns
import pandas as pd
from glob import glob
from tensorflow.keras.preprocessing import image as krs_image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_file = '/content/drive/Othercomputers/My Computer/Computer vision/projects/dl and cnns/classification apparels/train.zip'
test_file = '/content/drive/Othercomputers/My Computer/Computer vision/projects/dl and cnns/classification apparels/test.zip'

dir = './'
zip_ref = zipfile.ZipFile(train_file, 'r')
zip_ref.extractall(dir)
zip_ref = zipfile.ZipFile(test_file, 'r')
zip_ref.extractall(dir)
zip_ref.close()

In [4]:
train = pd.read_csv('./train.csv', dtype=str)
test = pd.read_csv('./test.csv', dtype=str)
def append_ext(fn):
    return fn+".png"
train["id"]=train["id"].apply(append_ext)
test["id"]=test["id"].apply(append_ext)

In [5]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
).flow_from_dataframe(
    dataframe=train,
    directory='./train',
    x_col='id',
    y_col='label',
    batch_size=128,
    class_mode='categorical',
    target_size=(28, 28)
)

Found 60000 validated image filenames belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [15]:
temp = []
for img_name in test.id:
  img_path = './test/' + img_name
  img = cv2.imread(img_path)
  img = img.astype('float32')
  temp.append(img)
test_x = np.stack(temp)
# test_x = temp
test_x /= 255.0
# test_x = test_x.reshape(-1, 784).astype('float32')
print(test_x.shape)

(10000, 28, 28, 3)


In [6]:
images, labels = train_generator.next()
print(images[0].size)

2352


In [7]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0

In [8]:
model.fit(train_generator, epochs=5, verbose=1)

Epoch 1/5
469/469 [==============================] - 57s 97ms/step - loss: 1.0620 - accuracy: 0.6043
Epoch 2/5
469/469 [==============================] - 45s 97ms/step - loss: 0.7411 - accuracy: 0.7211
Epoch 3/5
469/469 [==============================] - 44s 95ms/step - loss: 0.6565 - accuracy: 0.7512
Epoch 4/5
469/469 [==============================] - 45s 95ms/step - loss: 0.6000 - accuracy: 0.7740
Epoch 5/5
469/469 [==============================] - 46s 98ms/step - loss: 0.5674 - accuracy: 0.7874


In [16]:
preds = model.predict(test_x)

In [21]:
sample_submission = pd.DataFrame(columns=['id', 'label'])
sample_submission.id = test.id
sample_submission.label = np.argmax(preds, axis=1)
sample_submission.to_csv('test.csv')